In [1]:
import pandas as pd
import datetime as dt

'''
dfServices2020_2022 = pd.read_csv('Services2020_2022.csv')
dfServices2022_2024 = pd.read_csv('Services2022_2024.csv')

dfServices = pd.concat([dfServices2020_2022, dfServices2022_2024])
'''
dfServices = pd.read_csv('Services2024.csv')
dfSuccess = pd.read_csv('Successes.csv')

dfServices['Service Date'] = pd.to_datetime(dfServices['Service Date'])
dfSuccess['Exit Date'] = pd.to_datetime(dfSuccess['Exit Date'])

/var/folders/nm/sgb0qrh5259964c0_5jt6zfh0000gn/T/ipykernel_86337/4172125405.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/var/folders/nm/sgb0qrh5259964c0_5jt6zfh0000gn/T/ipykernel_86337/4172125405.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfServices['Service Date'] = pd.to_datetime(dfServices['Service Date'])
/var/folders/nm/sgb0qrh5259964c0_5jt6zfh0000gn/T/ipykernel_86337/4172125405.py:14: UserWarning: Could not infer format, so each element 

In [2]:
Housing = [
    'Permanent housing (other than RRH) for formerly homeless persons',
    'Owned by client, no ongoing housing subsidy',
    'Rental by client, no ongoing housing subsidy',
    'Rental by client, VASH housing Subsidy',
    'Rental by client, other ongoing housing subsidy',
    'Rental by client in a public housing unit',
    'Rental by client with RRH or equivalent subsidy',
    'Long-term care facility or nursing home',
    'Residential project or halfway house with no homeless criteria',
    'Owned by client, with ongoing housing subsidy',
    'Rental by client, with HCV voucher (tenant or project based)',
    'Rental by client, with GPD TIP housing subsidy'
    ]


# Date Ranges
serviceStartDate = dt.datetime(2024, 1, 1)
successStartDate = dt.datetime(2024, 1, 1)
endDate = dt.datetime(2024, 10, 1)
yearString = '2024'

# After services with SALT, how many clients transitioned into housing though any orgs

dfServices1 = dfServices.copy()

dfServices1 = dfServices1[dfServices1['Created by Org']=='SALT Outreach Inc.']

dfServicesInTimeframe = dfServices1[(dfServices1['Service Date'] >= serviceStartDate) & (dfServices1['Service Date'] <= endDate)]

dfSuccessInTimeframe = dfSuccess[(dfSuccess['Exit Date'] >= successStartDate) & (dfSuccess['Exit Date'] <= endDate)]

dfServices1 = dfServices1.sort_values('Service Date').drop_duplicates('Client ID', keep='first')

dfSuccess1 = dfSuccessInTimeframe[dfSuccessInTimeframe['Exit Destination'].isin(Housing)]
dfSuccess1 = dfSuccess1.sort_values('Exit Date').drop_duplicates('Client ID', keep='last')

dfIndirectSuccess = pd.merge(dfServices1, dfSuccess1, on='Client ID', how='inner')

dfIndirectSuccess['Exit Date'] = pd.to_datetime(dfIndirectSuccess['Exit Date'])
dfIndirectSuccess['Service Date'] = pd.to_datetime(dfIndirectSuccess['Service Date'])

dfIndirectSuccess = dfIndirectSuccess[dfIndirectSuccess['Exit Date'] > dfIndirectSuccess['Service Date']]
dfIndirectSuccess = dfIndirectSuccess[dfIndirectSuccess['Exit Date'] - dfIndirectSuccess['Service Date'] <= pd.Timedelta(365, 'D')]

print('After services with SALT, how many clients transitioned into housing though any orgs in ' + yearString)
print(dfIndirectSuccess['Client ID'].nunique(), 'clients transitioned into housing through any orgs in ' + yearString)
print()

# Number of clients who were directly housed by our case management team

dfSuccessInTimeframe = dfSuccess[(dfSuccess['Exit Date'] >= successStartDate) & (dfSuccess['Exit Date'] <= endDate)]

dfSuccess2 = dfSuccessInTimeframe[dfSuccessInTimeframe['Exit Destination'].isin(Housing)]

dfSuccess2 = dfSuccess2[dfSuccess2['Agency']=='SALT Outreach Inc.']

print('Number of clients who were directly housed by our case management team in ' + yearString)
print(dfSuccess2['Client ID'].nunique(), 'clients were directly housed by our case management team in ' + yearString)
print()

# Percent of people served that transitioned to case management

dfServices3 = dfServices.copy()

dfServices3 = dfServices3[dfServices3['Created by Org']=='SALT Outreach Inc.']

dfServicesInTimeframe = dfServices3[(dfServices3['Service Date'] >= serviceStartDate) & (dfServices3['Service Date'] <= endDate)]

dfServicesInTimeframe = dfServicesInTimeframe.sort_values('Service Date').drop_duplicates('Client ID', keep='first')

dfCaseManagement = dfServices3[dfServices3['Service']=='Case Management']

dfCaseManagement = dfCaseManagement[(dfCaseManagement['Service Date'] >= serviceStartDate) & (dfCaseManagement['Service Date'] <= endDate)]

dfCaseManagement = dfCaseManagement.rename(columns={'Service Date': 'Case Management Date'})

dfMerge = pd.merge(dfServicesInTimeframe, dfCaseManagement, on='Client ID', how='inner')

dfMerge['Case Management Date'] = pd.to_datetime(dfMerge['Case Management Date'])
dfMerge['Service Date'] = pd.to_datetime(dfMerge['Service Date'])

dfMerge = dfMerge[dfMerge['Case Management Date'] > dfMerge['Service Date']]

print('Percent of people served in ' + yearString + ' that transitioned to case management')
print(round((dfMerge['Client ID'].nunique() / dfServicesInTimeframe['Client ID'].nunique())*100, 2), '% of people served in ' + yearString + ' transitioned to case management')
print(round((dfMerge['Client ID'].nunique() / dfCaseManagement['Client ID'].nunique())*100, 2), '% of people served by case management in ' + yearString + ' transitioned came from other services')
print()

# After services with SALT, how many clients transitioned into housing though any orgs

dfServices1 = dfServices.copy()

dfServices1 = dfServices1[dfServices1['Created by Org']=='SALT Outreach Inc.']

dfServicesInTimeframe = dfServices1[(dfServices1['Service Date'] >= serviceStartDate) & (dfServices1['Service Date'] <= endDate)]

dfSuccessInTimeframe = dfSuccess[(dfSuccess['Exit Date'] >= successStartDate) & (dfSuccess['Exit Date'] <= endDate)]

dfServices1 = dfServices1.sort_values('Service Date').drop_duplicates('Client ID', keep='first')

dfSuccess1 = dfSuccessInTimeframe[dfSuccessInTimeframe['Exit Destination'].isin(Housing)]
dfSuccess1 = dfSuccess1.sort_values('Exit Date').drop_duplicates('Client ID', keep='last')

dfIndirectSuccess = pd.merge(dfServices1, dfSuccess1, on='Client ID', how='inner')

dfIndirectSuccess['Exit Date'] = pd.to_datetime(dfIndirectSuccess['Exit Date'])
dfIndirectSuccess['Service Date'] = pd.to_datetime(dfIndirectSuccess['Service Date'])

dfIndirectSuccess = dfIndirectSuccess[dfIndirectSuccess['Exit Date'] > dfIndirectSuccess['Service Date']]
dfIndirectSuccess = dfIndirectSuccess[dfIndirectSuccess['Exit Date'] - dfIndirectSuccess['Service Date'] <= pd.Timedelta(365, 'D')]

print('After services with SALT, how many clients transitioned into housing though any orgs in ' + yearString)
print(dfIndirectSuccess['Client ID'].nunique(), 'clients transitioned into housing through any orgs in ' + yearString)
print()

# Number of clients who were directly housed by our case management team 

dfSuccessInTimeframe = dfSuccess[(dfSuccess['Exit Date'] >= successStartDate) & (dfSuccess['Exit Date'] <= endDate)]

dfSuccess2 = dfSuccessInTimeframe[dfSuccessInTimeframe['Exit Destination'].isin(Housing)]

dfSuccess2 = dfSuccess2[dfSuccess2['Agency']=='SALT Outreach Inc.']

print('Number of clients who were directly housed by our case management team in ' + yearString)
print(dfSuccess2['Client ID'].nunique(), 'clients were directly housed by our case management team in ' + yearString)
print()

# Percent of people served in this year that transitioned to case management

dfServices4 = dfServices.copy()

dfServices4 = dfServices4[dfServices4['Created by Org']=='SALT Outreach Inc.']

dfServicesInTimeframe = dfServices4[(dfServices4['Service Date'] >= serviceStartDate) & (dfServices4['Service Date'] <= endDate)]

dfServicesInTimeframe = dfServicesInTimeframe.sort_values('Service Date').drop_duplicates('Client ID', keep='first')

dfCaseManagement = dfServices4[dfServices4['Service']=='Case Management']

dfCaseManagement = dfCaseManagement[(dfCaseManagement['Service Date'] >= serviceStartDate) & (dfCaseManagement['Service Date'] <= endDate)]

dfCaseManagement = dfCaseManagement.rename(columns={'Service Date': 'Case Management Date'})

dfMerge = pd.merge(dfServicesInTimeframe, dfCaseManagement, on='Client ID', how='inner')

dfMerge['Case Management Date'] = pd.to_datetime(dfMerge['Case Management Date'])
dfMerge['Service Date'] = pd.to_datetime(dfMerge['Service Date'])

dfMerge = dfMerge[dfMerge['Case Management Date'] > dfMerge['Service Date']]

print('Percent of people served in ' + yearString + ' that transitioned to case management')
print(round((dfMerge['Client ID'].nunique() / dfServicesInTimeframe['Client ID'].nunique())*100, 2), '% of people served in ' + yearString + ' transitioned to case management')
print(round((dfMerge['Client ID'].nunique() / dfCaseManagement['Client ID'].nunique())*100, 2), '% of people served by case management in ' + yearString + ' transitioned came from other services')
print()

'''
# How many clients were housed through any orgs after services from SALT, Q1 2024

dfServices5 = dfServices.copy()

dfServices5 = dfServices5[dfServices5['Created by Org']=='SALT Outreach Inc.']

dfServicesInTimeframe = dfServices5[(dfServices5['Service Date'] >= dt.datetime(2023, 1, 1)) & (dfServices5['Service Date'] < dt.datetime(2024, 3, 1))]

dfSuccessInTimeframe = dfSuccess[(dfSuccess['Exit Date'] >= dt.datetime(2024, 1, 1)) & (dfSuccess['Exit Date'] < dt.datetime(2024, 3, 1))]

dfServices5 = dfServices5.sort_values('Service Date').drop_duplicates('Client ID', keep='first')

dfSuccess5 = dfSuccessInTimeframe[dfSuccessInTimeframe['Exit Destination'].isin(Housing)]
dfSuccess5 = dfSuccess5.sort_values('Exit Date').drop_duplicates('Client ID', keep='last')

dfIndirectSuccess = pd.merge(dfServices5, dfSuccess5, on='Client ID', how='inner')

dfIndirectSuccess['Exit Date'] = pd.to_datetime(dfIndirectSuccess['Exit Date'])
dfIndirectSuccess['Service Date'] = pd.to_datetime(dfIndirectSuccess['Service Date'])

dfIndirectSuccess = dfIndirectSuccess[dfIndirectSuccess['Exit Date'] > dfIndirectSuccess['Service Date']]
dfIndirectSuccess = dfIndirectSuccess[dfIndirectSuccess['Exit Date'] - dfIndirectSuccess['Service Date'] <= pd.Timedelta(365, 'D')]

print('After services with SALT, how many clients transitioned into housing though any orgs in Q1 2024')
print(dfIndirectSuccess['Client ID'].nunique(), 'clients transitioned into housing through any orgs in Q1 2024')
print()
'''

After services with SALT, how many clients transitioned into housing though any orgs in 2024
64 clients transitioned into housing through any orgs in 2024

Number of clients who were directly housed by our case management team in 2024
26 clients were directly housed by our case management team in 2024

Percent of people served in 2024 that transitioned to case management
11.5 % of people served in 2024 transitioned to case management
73.38 % of people served by case management in 2024 transitioned came from other services

After services with SALT, how many clients transitioned into housing though any orgs in 2024
64 clients transitioned into housing through any orgs in 2024

Number of clients who were directly housed by our case management team in 2024
26 clients were directly housed by our case management team in 2024

Percent of people served in 2021 that transitioned to case management
11.5 % of people served in 2021 transitioned to case management
73.38 % of people served by case 

"\n# How many clients were housed through any orgs after services from SALT, Q1 2024\n\ndfServices5 = dfServices.copy()\n\ndfServices5 = dfServices5[dfServices5['Created by Org']=='SALT Outreach Inc.']\n\ndfServicesInTimeframe = dfServices5[(dfServices5['Service Date'] >= dt.datetime(2023, 1, 1)) & (dfServices5['Service Date'] < dt.datetime(2024, 3, 1))]\n\ndfSuccessInTimeframe = dfSuccess[(dfSuccess['Exit Date'] >= dt.datetime(2024, 1, 1)) & (dfSuccess['Exit Date'] < dt.datetime(2024, 3, 1))]\n\ndfServices5 = dfServices5.sort_values('Service Date').drop_duplicates('Client ID', keep='first')\n\ndfSuccess5 = dfSuccessInTimeframe[dfSuccessInTimeframe['Exit Destination'].isin(Housing)]\ndfSuccess5 = dfSuccess5.sort_values('Exit Date').drop_duplicates('Client ID', keep='last')\n\ndfIndirectSuccess = pd.merge(dfServices5, dfSuccess5, on='Client ID', how='inner')\n\ndfIndirectSuccess['Exit Date'] = pd.to_datetime(dfIndirectSuccess['Exit Date'])\ndfIndirectSuccess['Service Date'] = pd.to_date